In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2025-02-16 00:00:00')

current_date=Timestamp('2025-02-16 16:00:00')


In [6]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-02-16 16:12:42,129 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 16:12:42,129 INFO: Initializing external client
2025-02-16 16:12:42,138 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 16:12:43,271 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632
2025-02-16 16:12:45,878 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-16 16:12:45,881 INFO: Initializing external client
2025-02-16 16:12:45,881 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-16 16:12:46,958 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213632
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.38s) 


AssertionError: Time-series data is not complete. Make sure your feature pipeline is up and runnning.

In [ ]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

In [ ]:
predictions['pickup_hour'] = current_date
predictions

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(predictions, write_options={"wait_for_job": False})